# Credit Risk Assessment: Baseline Logistic Regression Model

---

### Import Libraries and set Configurations

In [1]:
import warnings, json
from datetime import date
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
import scipy.stats as ss
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    classification_report, confusion_matrix,
    precision_recall_curve, RocCurveDisplay, precision_score, recall_score, f1_score, PrecisionRecallDisplay
)
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
import category_encoders as ce
from scipy.stats import ks_2samp, loguniform, uniform
from sklearn import set_config

set_config(transform_output="default") 
data = Path("../data/processed")
random_state=42
warnings.filterwarnings("ignore")
stamp = date.today().isoformat()
models = Path("../models")
reports = Path("../reports")
figs = Path("../reports/figs")

### Load Data and Time Based Train/Test Split

In [2]:
df = pd.read_parquet("../data/processed/lc_cleaned.parquet")
cutoff_train = pd.to_datetime("2016-12-31")
cutoff_val = pd.to_datetime("2017-12-31")
train_idx = df["issue_d"] <= cutoff_train
val_idx = (df["issue_d"] > cutoff_train) & (df["issue_d"] <= cutoff_val)
test_idx= df["issue_d"] > cutoff_val
X_train, y_train = df.loc[train_idx].drop("default",axis=1), df.loc[train_idx,"default"]
X_val, y_val = df.loc[val_idx].drop("default",axis=1), df.loc[val_idx,"default"]
X_test,y_test = df.loc[test_idx].drop("default",axis=1), df.loc[test_idx, "default"]

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)
print("Test shape :", X_test.shape)
print("Train class balance:\n", y_train.value_counts(normalize=True))
print("Validation class balance:\n", y_val.value_counts(normalize=True))
print("Test class balance:\n", y_test.value_counts(normalize=True))

Train shape: (1132562, 186)
Validation shape: (181728, 186)
Test shape : (68061, 186)
Train class balance:
 default
0    0.795668
1    0.204332
Name: proportion, dtype: float64
Validation class balance:
 default
0    0.716191
1    0.283809
Name: proportion, dtype: float64
Test class balance:
 default
0    0.697081
1    0.302919
Name: proportion, dtype: float64


### Drop Datetime

In [3]:
dt_cols = X_train.select_dtypes("datetime64[ns]").columns          
X_train = X_train.drop(columns=dt_cols)
X_val = X_val.drop(columns=dt_cols)
X_test= X_test.drop(columns=dt_cols)

### Metrics

In [4]:
def ks_score(y, p):                   
    return ks_2samp(p[y == 0], p[y == 1]).statistic

def profit_curve(y, p, gain_tp=.80, cost_fp=.10):
    prec, rec, thr = precision_recall_curve(y, p)
    tp = rec * y.sum()
    fp = (tp / np.clip(prec, 1e-9, 1.0)) - tp
    profit = (gain_tp * tp - cost_fp * fp) / len(y)
    idx = profit.argmax()
    return float(thr[idx]), float(profit[idx])

def best_f1(y, p):
    prec, rec, thr = precision_recall_curve(y, p)
    f1 = 2 * prec * rec / (prec + rec + 1e-12)
    return float(thr[f1.argmax()])

def metrics(y, p, thr):
    ŷ = (p > thr).astype(int)
    return dict(
        AUC = roc_auc_score(y, p),
        PR_AUC= average_precision_score(y, p),
        KS = ks_score(y, p),
        Precision = precision_score(y, ŷ),
        Recall= recall_score(y, ŷ),
        ConfMatrix = confusion_matrix(y, ŷ).tolist(),
        ClassReport= classification_report(y, ŷ, digits=3, zero_division=0)
    )

### Preprocessing

In [5]:
cat_all = X_train.select_dtypes("category").columns.tolist()
num_all = X_train.select_dtypes(np.number).columns.tolist()

low_card = [c for c in cat_all if X_train[c].nunique() <= 6]
high_card= [c for c in cat_all if c not in low_card]

num_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

ohe_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output = True, min_frequency=0.005))
])

woe_pipe = ce.WOEEncoder(handle_missing="value", 
    handle_unknown="value", random_state=42,sigma=0.02,drop_invariant=True, regularization = 0.05, verbose=0)

X_train[high_card] = woe_pipe.fit_transform(X_train[high_card], y_train)
X_val[high_card] = woe_pipe.transform(X_val[high_card])
X_test[high_card] = woe_pipe.transform(X_test[high_card])
joblib.dump(woe_pipe, models/"woe_encoder.joblib")

preprocessor = ColumnTransformer([
    ("num", num_pipe, num_all),
    ("lowc", ohe_pipe, low_card)
], remainder="passthrough")

print("LR train shape:", X_train.shape)
print("LR val shape:", X_val.shape)
print("LR test shape:", X_test.shape)

LR train shape: (1132562, 183)
LR val shape: (181728, 183)
LR test shape: (68061, 183)


### Save Objects

In [6]:
processed = Path("../data/processed"); processed.mkdir(exist_ok=True)
X_train.to_pickle(processed/"X_train_lr.pkl")
X_val.to_pickle(processed/"X_val_lr.pkl")
X_test.to_pickle(processed/"X_test_lr.pkl")
pd.Series(y_train).to_csv(processed/"y_train_lr.csv", index=False)
pd.Series(y_val).to_csv(processed/"y_val_lr.csv", index=False)
pd.Series(y_test).to_csv(processed/"y_test_lr.csv", index=False)
joblib.dump(preprocessor, "../models/lr_preprocessor.joblib")
print("saved to", processed.resolve())

saved to /Users/pranavrao/Documents/ai-ml-projects/github-repos/aura-xai-finrisk-llm/data/processed


### Train Logistic Regression Model

In [7]:
model_version = "v1"
logreg_path = models / f"logreg_{model_version}.joblib"
meta_path = models / f"logreg_meta_{model_version}.json"

if logreg_path.exists():
    print("Using cached Logistic Regression model")
    logreg = joblib.load(logreg_path)
    with open(meta_path) as fp:
        meta = json.load(fp)
else:
    print("No cached model found, training new Logistic Regression model with CV")
    tscv = TimeSeriesSplit(n_splits=3)
    base_lr = LogisticRegression(
        penalty='elasticnet',
        max_iter=2000,
        l1_ratio=0.5,
        solver="saga",
        class_weight="balanced",
        random_state=random_state,
        n_jobs=1,
        tol = 5e-4,
        verbose=2
    )

    pipeline = Pipeline([
    ("pre", preprocessor),
    ("clf", base_lr)
    ], memory=joblib.Memory("./cache"))

    param_dist = {
    "clf__C": ss.loguniform(5e-2, 5),
    "clf__l1_ratio": ss.uniform(0, 1)     
    }

    sample_mask = y_train.sample(frac=0.1, random_state=42).index
    X_search, y_search = X_train.loc[sample_mask], y_train.loc[sample_mask]
    
    search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_dist,
    n_iter=20,                
    scoring="roc_auc",
    cv=tscv,
    n_jobs=3,
    verbose=2,
    random_state=42,
    return_train_score=False
    ).fit(X_search, y_search)
    best_C, best_l1 = search.best_params_["clf__C"], search.best_params_["clf__l1_ratio"]
    print("best AUC (CV):", search.best_score_)
    print("best C:", search.best_params_["clf__C"])
    print("best l1_ratio:", search.best_params_["clf__l1_ratio"])
  

    final_lr = LogisticRegression(
        penalty="elasticnet", solver="saga",
        C=best_C, l1_ratio=best_l1,
        max_iter=2000, tol=5e-4,
        class_weight="balanced", n_jobs=1,
        random_state=random_state, verbose = 2
    )
    final_pipe = Pipeline(
        [("pre", preprocessor), ("clf", final_lr)],
        memory=joblib.Memory("./cache")
    ).fit(X_train, y_train)

    cal = CalibratedClassifierCV(
        estimator=final_pipe, method="isotonic", cv="prefit"
    ).fit(X_val, y_val)

    print("Model Fitted")
    joblib.dump(cal, logreg_path)

    meta = {
        "date": date.today().isoformat(),
        "best_C" : float(search.best_params_["clf__C"]),
        "best_l1": float(search.best_params_["clf__l1_ratio"]),
        "solver": base_lr.solver,
        "penalty": base_lr.penalty,
        "class_weight": base_lr.class_weight,
        "cv_folds": 3,
        "scoring": "roc_auc",
        "max_iter": base_lr.max_iter,
        "random_state": random_state,
        "Type": "Logistic Regression"
    }
    with open(meta_path, "w") as fp:
        json.dump(meta, fp, indent=2)

    print("Model and metadata saved:", logreg_path.name, meta_path.name)

No cached model found, training new Logistic Regression model with CV
Fitting 3 folds for each of 20 candidates, totalling 60 fits
Epoch 1, change: 1
Epoch 2, change: 0.11020163
Epoch 3, change: 0.086729903
Epoch 1, change: 1
Epoch 4, change: 0.038428533
Epoch 5, change: 0.031013628
Epoch 2, change: 0.22023747
Epoch 6, change: 0.025300584
Epoch 1, change: 1
Epoch 7, change: 0.023138688
Epoch 3, change: 0.088405309
Epoch 8, change: 0.023073844
Epoch 9, change: 0.021711109
Epoch 2, change: 0.21926392
Epoch 10, change: 0.021027562
Epoch 4, change: 0.057199978
Epoch 11, change: 0.020267485
Epoch 12, change: 0.019550018
Epoch 5, change: 0.042354828
Epoch 13, change: 0.018871859
Epoch 3, change: 0.097069584
Epoch 14, change: 0.018216891
Epoch 6, change: 0.034047022
Epoch 15, change: 0.017602817
Epoch 16, change: 0.016928005
Epoch 4, change: 0.055182647
Epoch 17, change: 0.016313664
Epoch 7, change: 0.027300843
Epoch 18, change: 0.015760153
Epoch 19, change: 0.01518724
Epoch 8, change: 0.0238

/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['revol_bal_joint' 'sec_app_fico_range_low' 'sec_app_fico_range_high'
 'sec_app_inq_last_6mths' 'sec_app_mort_acc' 'sec_app_open_acc'
 'sec_app_revol_util' 'sec_app_open_act_il' 'sec_app_num_rev_accts'
 'sec_app_chargeoff_within_12_mths' 'sec_app_collections_12_mths_ex_med'
 'sec_app_mths_since_last_major_derog']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Epoch 144, change: 0.0032829682
Epoch 222, change: 0.0018390176
Epoch 1, change: 1
Epoch 223, change: 0.0018260007
Epoch 2, change: 0.11088802
Epoch 145, change: 0.0032570475
Epoch 3, change: 0.086632331
Epoch 224, change: 0.0018129273
Epoch 4, change: 0.040198073
Epoch 5, change: 0.031198297
Epoch 225, change: 0.0018034487
Epoch 146, change: 0.0032303637
Epoch 6, change: 0.026625207
Epoch 7, change: 0.02446901
Epoch 226, change: 0.0017906378
Epoch 8, change: 0.023921463
Epoch 9, change: 0.022938307
Epoch 147, change: 0.0032047595
Epoch 227, change: 0.001779547
Epoch 10, change: 0.022207736
Epoch 11, change: 0.021406955
Epoch 228, change: 0.001768597
Epoch 12, change: 0.02064629
Epoch 148, change: 0.003179885
Epoch 13, change: 0.019923787
Epoch 229, change: 0.0017570143
Epoch 14, change: 0.019238474
Epoch 15, change: 0.018585734
Epoch 149, change: 0.0031535373
Epoch 230, change: 0.0017454898
Epoch 16, change: 0.017875164
Epoch 17, change: 0.01722273
Epoch 231, change: 0.0017341259
Epoc

/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['revol_bal_joint' 'sec_app_fico_range_low' 'sec_app_fico_range_high'
 'sec_app_inq_last_6mths' 'sec_app_mort_acc' 'sec_app_open_acc'
 'sec_app_revol_util' 'sec_app_open_act_il' 'sec_app_num_rev_accts'
 'sec_app_chargeoff_within_12_mths' 'sec_app_collections_12_mths_ex_med'
 'sec_app_mths_since_last_major_derog']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Epoch 310, change: 0.0010389599
Epoch 496, change: 0.00097824642
Epoch 497, change: 0.00097625704
Epoch 498, change: 0.00097482631
Epoch 311, change: 0.001033459
Epoch 1, change: 1
Epoch 499, change: 0.00097380471
Epoch 500, change: 0.00097254324
Epoch 2, change: 0.2203708
Epoch 501, change: 0.00096988164
Epoch 312, change: 0.0010273713
Epoch 502, change: 0.0009686952
Epoch 3, change: 0.088551591
Epoch 503, change: 0.00096796917
Epoch 313, change: 0.0010215311
Epoch 504, change: 0.00096634095
Epoch 4, change: 0.057331374
Epoch 505, change: 0.00096387545
Epoch 506, change: 0.00096288871
Epoch 314, change: 0.0010158126
Epoch 5, change: 0.042465231
Epoch 507, change: 0.00096101445
Epoch 508, change: 0.00095979159
Epoch 6, change: 0.034157422
Epoch 509, change: 0.00095728331
Epoch 315, change: 0.0010098323
Epoch 510, change: 0.00095703329
Epoch 7, change: 0.0273977
Epoch 511, change: 0.00095519423
Epoch 316, change: 0.0010039218
Epoch 512, change: 0.00095438306
Epoch 8, change: 0.023969407

...

Epoch 313, change: 0.0011237589
Epoch 12, change: 0.013077935
Epoch 314, change: 0.0011182841
Epoch 13, change: 0.011684949
Epoch 315, change: 0.0011122425
Epoch 316, change: 0.0011057977
Epoch 14, change: 0.010492546
Epoch 317, change: 0.0010992748
Epoch 15, change: 0.0095016886
Epoch 318, change: 0.0010932884
Epoch 319, change: 0.0010867701
Epoch 16, change: 0.0086738666
Epoch 320, change: 0.00108109
Epoch 17, change: 0.0081546433
Epoch 321, change: 0.0010746326
Epoch 322, change: 0.0010685113
Epoch 18, change: 0.007744391
Epoch 323, change: 0.0010623577
Epoch 19, change: 0.0074211674
Epoch 324, change: 0.0010568658
Epoch 325, change: 0.0010513188
Epoch 20, change: 0.0070165684
Epoch 326, change: 0.0010455009
Epoch 21, change: 0.0067261434
Epoch 327, change: 0.0010396609
Epoch 328, change: 0.0010335898
Epoch 22, change: 0.0064734718
Epoch 329, change: 0.0010288357
Epoch 23, change: 0.0061230815
Epoch 330, change: 0.001022487
Epoch 331, change: 0.001016589
Epoch 24, change: 0.00593403

/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['revol_bal_joint' 'sec_app_fico_range_low' 'sec_app_fico_range_high'
 'sec_app_inq_last_6mths' 'sec_app_mort_acc' 'sec_app_open_acc'
 'sec_app_revol_util' 'sec_app_open_act_il' 'sec_app_num_rev_accts'
 'sec_app_chargeoff_within_12_mths' 'sec_app_collections_12_mths_ex_med'
 'sec_app_mths_since_last_major_derog']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Epoch 116, change: 0.0042373409
Epoch 117, change: 0.0041999193
Epoch 118, change: 0.0041607194
Epoch 119, change: 0.0041219154
Epoch 120, change: 0.0040879837
Epoch 121, change: 0.0040503393
Epoch 122, change: 0.004014428
Epoch 123, change: 0.0039792998
Epoch 124, change: 0.0039435743
Epoch 125, change: 0.0039096406
Epoch 126, change: 0.0038767572
Epoch 127, change: 0.0038409677
Epoch 128, change: 0.0038090918
Epoch 129, change: 0.0037779396
Epoch 130, change: 0.0037446499
Epoch 131, change: 0.0037118155
Epoch 132, change: 0.0036808221
Epoch 133, change: 0.0036481474
Epoch 134, change: 0.003618398
Epoch 135, change: 0.0035894323
Epoch 136, change: 0.0035566086
Epoch 137, change: 0.0035283267
Epoch 138, change: 0.003497501
Epoch 139, change: 0.0034696118
Epoch 140, change: 0.0034412824
Epoch 141, change: 0.0034123426
Epoch 142, change: 0.003382882
Epoch 143, change: 0.0033565974
Epoch 144, change: 0.0033297515
Epoch 145, change: 0.0033027581
Epoch 146, change: 0.0032757374
Epoch 147, c

/opt/anaconda3/lib/python3.12/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['revol_bal_joint' 'sec_app_fico_range_low' 'sec_app_fico_range_high'
 'sec_app_inq_last_6mths' 'sec_app_mort_acc' 'sec_app_open_acc'
 'sec_app_revol_util' 'sec_app_open_act_il' 'sec_app_num_rev_accts'
 'sec_app_chargeoff_within_12_mths' 'sec_app_collections_12_mths_ex_med'
 'sec_app_mths_since_last_major_derog']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


Epoch 1, change: 1
Epoch 2, change: 0.22304204
Epoch 3, change: 0.10832376
Epoch 4, change: 0.061153267
Epoch 5, change: 0.043106337
Epoch 6, change: 0.031605119
Epoch 7, change: 0.026356614
Epoch 8, change: 0.022064358
Epoch 9, change: 0.018990405
Epoch 10, change: 0.016655756
Epoch 11, change: 0.014723468
Epoch 12, change: 0.013065167
Epoch 13, change: 0.011701517
Epoch 14, change: 0.010567884
Epoch 15, change: 0.0096677163
Epoch 16, change: 0.0088309272
Epoch 17, change: 0.0080270711
Epoch 18, change: 0.0074078584
Epoch 19, change: 0.0070512965
Epoch 20, change: 0.0067847466
Epoch 21, change: 0.0065305649
Epoch 22, change: 0.0064459443
Epoch 23, change: 0.0065227555
Epoch 24, change: 0.0065912671
Epoch 25, change: 0.0066548457
Epoch 26, change: 0.0067184773
Epoch 27, change: 0.0067414913
Epoch 28, change: 0.0067652818
Epoch 29, change: 0.0067896754
Epoch 30, change: 0.0068011999
Epoch 31, change: 0.0068135152
Epoch 32, change: 0.0068159362
Epoch 33, change: 0.0068275822
Epoch 34, ch

### Evaluation Metics - Train/Test

In [8]:
p_train = cal.predict_proba(X_train)[:, 1]
p_val = cal.predict_proba(X_val)[:, 1]

thr_f1_train = best_f1(y_train, p_train)
thr_f1_val = best_f1(y_val, p_val)
thr_profit_val, _ = profit_curve(y_val, p_val)

m_train = metrics(y_train, p_train, thr_f1_train)
m_val_f = metrics(y_val, p_val, thr_f1_val)
m_val_p = metrics(y_val,p_val,thr_profit_val)

print("=== TRAIN (F1-opt) ===")
print(m_train["ClassReport"])
print("\n=== VALID (F1-opt) ===")
print(m_val_f["ClassReport"])
print("\n=== VALID (profit-opt) ===")
print(m_val_p["ClassReport"])

def save_curves(split:str, y_true, proba):
    roc = RocCurveDisplay.from_predictions(y_true, proba, name=f"LR – {split.upper()}")
    roc.figure_.savefig(figs / f"roc_lr_{split}_{model_version}.png", dpi=300)
    plt.close(roc.figure_)

    pr = PrecisionRecallDisplay.from_predictions(y_true, proba, name=f"LR – {split.upper()}")
    pr.figure_.savefig(figs/f"pr_lr_{split}_{model_version}.png", dpi=300)
    plt.close(pr.figure_)

save_curves("train", y_train, p_train)
save_curves("val",y_val,p_val)
print("ROC & PR curves saved for train + val")

md_path = reports / f"metrics_logreg_trainval_{model_version}.md"
with open(md_path, "w") as f:
    f.write(f"# Logistic Regression – {model_version}  \n")
    f.write(f"*Date generated:* {date.today().isoformat()}\n\n")

    # ---------- thresholds ----------
    f.write("## Thresholds chosen\n")
    f.write("| Split | F1-opt | Profit-opt |\n")
    f.write("|-------|-------:|-----------:|\n")
    f.write(f"| Train | {thr_f1_train:.3f} | – |\n")
    f.write(f"| Val | {thr_f1_val:.3f} | {thr_profit_val:.3f} |\n\n")

    # ---------- TRAIN ----------
    f.write("## Train (F1-optimised)\n")
    f.write(f"- **AUC:** `{m_train['AUC']:.4f}`  \n")
    f.write(f"- **PR-AUC:** `{m_train['PR_AUC']:.4f}`  \n")
    f.write(f"- **KS:** `{m_train['KS']:.4f}`  \n\n")
    f.write("```text\n" + m_train["ClassReport"] + "\n```\n\n")

    # ---------- VAL: F1 ----------
    f.write("## Validation (F1-optimised)\n")
    f.write(f"- **AUC:** `{m_val_f['AUC']:.4f}`  \n")
    f.write(f"- **PR-AUC:** `{m_val_f['PR_AUC']:.4f}`  \n")
    f.write(f"- **KS:** `{m_val_f['KS']:.4f}`  \n\n")
    f.write("```text\n" + m_val_f["ClassReport"] + "\n```\n\n")

    # ---------- VAL: Profit ----------
    f.write("## Validation (Profit-optimised)\n")
    f.write(f"- **AUC:** `{m_val_p['AUC']:.4f}`  \n")
    f.write(f"- **PR-AUC:** `{m_val_p['PR_AUC']:.4f}`  \n")
    f.write(f"- **KS:** `{m_val_p['KS']:.4f}`  \n\n")
    f.write("```text\n" + m_val_p['ClassReport'] + "\n```\n")

print("train & validation curves saved and markdown written to", md_path.relative_to(reports))

=== TRAIN (F1-opt) ===
              precision    recall  f1-score   support

           0      0.878     0.742     0.804    901143
           1      0.373     0.597     0.459    231419

    accuracy                          0.712   1132562
   macro avg      0.625     0.669     0.631   1132562
weighted avg      0.774     0.712     0.733   1132562


=== VALID (F1-opt) ===
              precision    recall  f1-score   support

           0      0.837     0.791     0.813    130152
           1      0.536     0.612     0.572     51576

    accuracy                          0.740    181728
   macro avg      0.687     0.701     0.692    181728
weighted avg      0.752     0.740     0.745    181728


=== VALID (profit-opt) ===
              precision    recall  f1-score   support

           0      0.932     0.256     0.401    130152
           1      0.336     0.953     0.497     51576

    accuracy                          0.453    181728
   macro avg      0.634     0.604     0.449    181728

### Evaluation Metrics - Test

In [9]:
p_test = cal.predict_proba(X_test)[:, 1]
thr_f1_test = best_f1(y_test, p_test)
thr_profit_test, _ = profit_curve(y_test, p_test)

m_test_f = metrics(y_test, p_test, thr_f1_test)
m_test_p = metrics(y_test, p_test, thr_profit_test)

print("=== TEST (F1-opt) ===")
print(m_test_f["ClassReport"])
print("\n=== TEST (profit-opt) ===")
print(m_test_p["ClassReport"])

roc = RocCurveDisplay.from_predictions(y_test, p_test, name="LR – TEST")
roc.figure_.savefig(figs / f"roc_lr_test_{model_version}.png", dpi=300)
plt.close(roc.figure_)

pr  = PrecisionRecallDisplay.from_predictions(y_test, p_test, name="LR – TEST")
pr.figure_.savefig(figs / f"pr_lr_test_{model_version}.png", dpi=300)
plt.close(pr.figure_)

md_path = reports / f"metrics_logreg_test_{model_version}.md"
with open(md_path, "a") as f:          
    f.write("\n---\n\n")
    f.write("## Test results\n")
    f.write("| Threshold type | Value |\n")
    f.write("|----------------|------:|\n")
    f.write(f"| F1-opt | {thr_f1_test:.3f} |\n")
    f.write(f"| Profit-opt | {thr_profit_test:.3f} |\n\n")

    # F1-optimised block
    f.write("### Test (F1-optimised)\n")
    f.write(f"- **AUC:** `{m_test_f['AUC']:.4f}`  \n")
    f.write(f"- **PR-AUC:** `{m_test_f['PR_AUC']:.4f}`  \n")
    f.write(f"- **KS:** `{m_test_f['KS']:.4f}`  \n\n")
    f.write("```text\n" + m_test_f["ClassReport"] + "\n```\n\n")

    # Profit-optimised block
    f.write("### Test (Profit-optimised)\n")
    f.write(f"- **AUC:** `{m_test_p['AUC']:.4f}`  \n")
    f.write(f"- **PR-AUC:** `{m_test_p['PR_AUC']:.4f}`  \n")
    f.write(f"- **KS:** `{m_test_p['KS']:.4f}`  \n\n")
    f.write("```text\n" + m_test_p['ClassReport'] + "\n```\n")

print("test curves saved and markdown updated to", md_path.relative_to(reports))

=== TEST (F1-opt) ===
              precision    recall  f1-score   support

           0      0.853     0.958     0.902     47444
           1      0.864     0.620     0.722     20617

    accuracy                          0.855     68061
   macro avg      0.858     0.789     0.812     68061
weighted avg      0.856     0.855     0.848     68061


=== TEST (profit-opt) ===
              precision    recall  f1-score   support

           0      0.935     0.423     0.582     47444
           1      0.412     0.932     0.572     20617

    accuracy                          0.577     68061
   macro avg      0.673     0.677     0.577     68061
weighted avg      0.776     0.577     0.579     68061

test curves saved and markdown updated to metrics_logreg_test_v1.md


In [ ]:
import wandb
from wandb.sklearn import plot_classifier
wandb.init(
    project="loan-default-prediction",
    name=f"logistic-regression-{model_version}",
    config={
        "penalty": "elasticnet",
        "solver": "saga",
        "C": best_C,
        "l1_ratio": best_l1,
        "cv_folds": 3,
        "random_state": random_state,
    }
)

